In [3]:
import sklearn
import sklearn.preprocessing as preprocessing
import pandas as pd

sklearn.set_config(transform_output="pandas")

In [22]:
# Pre-processing: Cateogry -> 0, 1, 2, 3 ..etc
test_data = pd.DataFrame({
    "Color" : ["A", "A", "B", "C"]
})

encoder = preprocessing.OrdinalEncoder()
encoder.fit(test_data)
test_data_tr = encoder.transform(test_data)
print(f"Categories: \n\t {encoder.categories_}")
print(f"Transformed: \n {test_data_tr}")

Categories: 
	 [array(['A', 'B', 'C'], dtype=object)]
Transformed: 
    Color
0    0.0
1    0.0
2    1.0
3    2.0
